In [1]:
import pandas as pd
import numpy as np
import glob
import os
import datetime
import re
from datetime import datetime

# %cd /Volumes/SEB_USB/Season_10/

In [ ]:
def get_date_from_string(text):
    match = re.search(r'\d{4}-\d{2}-\d{2}', text)
    date = datetime.strptime(match.group(), '%Y-%m-%d').date()
    return date


# --------------------------------------------------
def get_ps2_dataframe(csv_path):
    df_list = []

    for csv in glob.glob(csv_path):
        date_df = pd.read_csv(csv)
        date = get_date_from_string(os.path.basename(csv))
        date_df['date'] = pd.to_datetime(date)
        df_list.append(date_df)  

    ps2_df = pd.concat(df_list).drop('Unnamed: 0', axis=1)

    return ps2_df


# --------------------------------------------------
def clean_plot_column_rgb_flir(rgb_flir_merged):
    
    rgb_flir_merged['plot'] = rgb_flir_merged['plot'].str.split('_', expand=True)[6].astype(str).str.zfill(2) + rgb_flir_merged['plot'].str.split('_', expand=True)[8].astype(str).str.zfill(2)

    return rgb_flir_merged


# --------------------------------------------------
def clean_plot_column_ps2(ps2_df):

    ps2_df['plot'] = ps2_df['Plot'].str.split(' ', expand=True)[6].astype(str).str.zfill(2) + ps2_df['Plot'].str.split(' ', expand=True)[8].astype(str).str.zfill(2)
    ps2_df = ps2_df.drop('Plot', axis=1)

    return ps2_df

# RGB & FLIR

### Open data

In [ ]:
thermal = pd.read_csv('Haiquan/s10_flir.csv').drop('treatment', axis=1)
rgb = pd.read_csv('Haiquan/stereoTop_full_season_clustering.csv')

### Convert date string to datetime

In [ ]:
thermal['date'] = pd.to_datetime(thermal['date'])
rgb['date'] = pd.to_datetime(rgb['date'])

### Set shared index

In [ ]:
rgb = rgb.set_index(['date', 'plant_name'])
thermal = thermal.set_index(['date', 'plant_name'])

### Join RGB and FLIR

In [ ]:
rgb_flir_merged = rgb.join(thermal, how='outer').reset_index()

In [ ]:
rgb_flir_merged = clean_plot_column_rgb_flir(rgb_flir_merged)

# RGB|FLIR & PSII

### Open PSII data

In [ ]:
ps2_df = get_ps2_dataframe('./Haiquan/PS2/*.csv')
ps2_df = clean_plot_column_ps2(ps2_df)

### Prepare RGB|FLIR & PSII for merging

In [ ]:
ps2_df = ps2_df.set_index(['date', 'plot'])
rgb_flir_merged = rgb_flir_merged.set_index(['date', 'plot'])

In [ ]:
merged = rgb_flir_merged.join(ps2_df, how='outer').reset_index()

In [ ]:
merged.to_csv('season10_lettuce_gantry_data.csv')